# **YOLOv8 Training for UFDD dataset**

**Author:** [Yi-Jie Wong](https://www.linkedin.com/in/wongyijie/)<br>
**Date created:** 2023/07/19<br>
**Last modified:** 2023/10/28<br>
**Description:** Training YOLOv8 for UFDD Dataset + Conversion to OpenVINO

Reference
*   [How to use the Python API for YOLOv8](https://learnopencv.com/train-yolov8-on-custom-dataset/)
*   [How to use YOLOv5](https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb#scrollTo=zR9ZbuQCH7FX)
*   [yolo data format: x_c, y_c, w, h](https://github.com/ultralytics/yolov5/issues/2293#issuecomment-785534291)
*   [UFDD dataset](https://paperswithcode.com/dataset/ufdd)
*   [OpenVINO for YOLOv8](https://github.com/openvinotoolkit/openvino_notebooks/blob/main/notebooks/230-yolov8-optimization/230-yolov8-optimization.ipynb)



## **Get Ready UFDD dataset**

### Download UFDD dataset

Unconstrained Face Detection Dataset (UFDD) is a Face Detection dataset aims to fuel further research in unconstrained face detection.

<img src="https://production-media.paperswithcode.com/datasets/Screenshot_2021-02-01_at_16.05.53.png" alt="UFDD data example">

In [ ]:
from IPython.display import clear_output

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1aGR7FryrRuS86S9LBAqFksy-QDqsgBRV' -O "UFDD-annotationfile.zip"
!unzip "UFDD-annotationfile.zip"

clear_output()

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1bZGzDx_CgNnxoRdLnmMLiZ3F9k5lnY4J' -O "UFDD_information.zip"
!unzip "UFDD_information.zip"

clear_output()

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1o-lsXB7XLc4F39zQyZgwrabWyN1M5NBY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1o-lsXB7XLc4F39zQyZgwrabWyN1M5NBY" -O "UFDD_val.zip" && rm -rf /tmp/cookies.txt
!unzip "UFDD_val.zip"

clear_output()

In [ ]:
# remove the zip file
import os, shutil

for filename in os.listdir(os.getcwd()):
    if filename.endswith('.zip'):
        os.remove(filename)

### Get ready the dataset
1.   Split dataset
2.   Convert dataset into [appropriate format](https://github.com/ultralytics/yolov5/issues/2293#issuecomment-785534291)


#### 1. Split Dataset
For YOLOv5 and YOLOv8, you need to split your dataset following the directory structure below:
```
UFDD
├── train
│   └── images  
│   └── labels  
├── valid
│   └── images  
│   └── labels  
```
In this case, we created a folder called ```UFDD``` to store all data for UFDD dataset. All training data and validation data is stored in ```train``` and ```valid``` folder, respectively. ```images``` store all image for training/validation, while ```labels``` store all the labels for each image. Refer next part for the label format </br></br>



#### 2. Dataset format
For each ```xxx.jpg``` (or other image format), you have a corresponding label file ```xxx.txt``` which follows the format below:

*   One row per object
*   Each row is ```class, x_center, y_center, width, height``` format.
*   Box coordinates must be in ```normalized xywh format (from 0 - 1)```. If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
*   Class numbers are zero-indexed (start from 0).

In [ ]:
data_dir = 'UFDD'

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
    os.mkdir(os.path.join(data_dir, 'train'))
    os.mkdir(os.path.join(data_dir, 'train', 'images'))
    os.mkdir(os.path.join(data_dir, 'train', 'labels'))

    os.mkdir(os.path.join(data_dir, 'valid'))
    os.mkdir(os.path.join(data_dir, 'valid', 'images'))
    os.mkdir(os.path.join(data_dir, 'valid', 'labels'))

In [ ]:
import matplotlib.pyplot as plt

filename = 'UFDD-annotationfile/UFDD_split/UFDD_val_bbx_gt.txt'
#filename = 'UFDD-annotationfile/UFDD_split/UFDD_val_bbx_gt-woDistractor.txt'

with open(filename, 'r') as file:
    # read lines
    lines = file.readlines()

    # loop all lines
    paths = []
    annots = []
    annot = []

    i = 0
    while i != len(lines):
        line = lines[i].replace('\n', '')
        path = os.path.join('UFDD_val/images', line)
        if os.path.exists(path):
            # add path to paths
            paths.append(path)
            i += 1

            # load image
            img = plt.imread(path)
            try:
                y_shape, x_shape, _ = img.shape
            except:
                y_shape, x_shape = img.shape

            # next line is total object in this image
            count = int(lines[i].replace('\n', ''))
            i += 1

            # get all annot in this image
            for _ in range(count):
                # read next line
                line = lines[i].replace('\n', '')

                # split to x_start, y_start, x_range, y_range
                alist = line.split()
                alist = [int(item) for item in alist]
                x_start, y_start, x_range, y_range, _, _, _, _, _ = alist

                # normalize to 0 - 1
                x_start, x_range = x_start / x_shape, x_range / x_shape
                y_start, y_range = y_start / y_shape, y_range / y_shape

                # find x_c, y_c, w, h
                x_c = x_start + x_range/2
                y_c = y_start + y_range/2
                w = x_range
                h = y_range

                annot.append('0 {:.6f} {:.6f} {:.6f} {:.6f}'.format(x_c, y_c, w, h))
                i += 1

            # add annot to annots
            annots.append(annot)
            annot = []
        else:
            raise NotImplementedError

assert len(paths) == len(annots)

In [ ]:
# show that the format is correct

i = 0

# get image
img = plt.imread(paths[i])
y_shape, x_shape, _ = img.shape

annot = annots[i][0]
alist = annot.split()
alist = [float(item) for item in alist]
_, x_c, y_c, w, h = alist
x_c, w = x_c * x_shape, w * x_shape
y_c, h = y_c * y_shape, h * y_shape
y_start = int(y_c - h / 2)
x_start = int(x_c - w / 2)
y_range = int(h)
x_range = int(w)

img = img[y_start:int(y_start+y_range), x_start:int(x_start+x_range),:]
plt.imshow(img)

In [ ]:
# shuffle the data
import numpy as np

SEED = 123

np.random.seed(SEED)
np.random.shuffle(paths)
np.random.seed(SEED)
np.random.shuffle(annots)

In [ ]:
# split the data (in appropriate format)

TRAIN_RATIO = 0.8

for i, (path, annot) in enumerate(zip(paths, annots)):
    if i <= int(TRAIN_RATIO * len(paths)):
        image_dir = 'UFDD/train/images'
        label_dir = 'UFDD/train/labels'
    else:
        image_dir = 'UFDD/valid/images'
        label_dir = 'UFDD/valid/labels'

    # get path
    ori_image_path = paths[i]
    new_image_path = os.path.join(image_dir, os.path.basename(paths[i]))
    annot_path = os.path.join(label_dir, os.path.splitext(os.path.basename(new_image_path))[0]+'.txt')

    # copy image to new directory
    shutil.copyfile(ori_image_path, new_image_path)

    # save annotation as txt file
    with open(annot_path, 'w') as file:
        for item in annot:
            file.write(item + '\n')

In [ ]:
# get ready the yaml file for the dataset
current_dir = os.getcwd()

yaml_config = [
    f"train: '{current_dir}/UFDD/train'",
    f"val: '{current_dir}/UFDD/valid'",
    "",
    "# class names",
    "names: ",
    "  0: 'face'"
]

# save annotation as txt file
yaml_file = 'UFDD.yaml'
with open(yaml_file, 'w') as file:
    for item in yaml_config:
        file.write(item + '\n')

## **YOLOv8 Training**

### Training

In [ ]:
!pip install ultralytics==8.0.146

In [ ]:
# !yolo task=detect \
# mode=predict \
# model=yolov8n.pt \
# conf=0.25 \
# source='https://media.roboflow.com/notebooks/examples/dog.jpeg'

In [ ]:
from ultralytics import YOLO

# Load the pretrained model
model = YOLO('yolov8m.pt')

In [ ]:
from ultralytics import YOLO

# Load the pretrained model
model = YOLO('yolov8m.pt')

# Training.
results = model.train(
   data=yaml_file,
   imgsz=640,
   epochs=200, # adjust accordingly
   batch=32, # use the largest batch size your machine can
   #amp=False,
   name='yolov8m_custom')

In [ ]:
# https://docs.ultralytics.com/modes/val/#key-features-of-val-mode
model = YOLO('/content/runs/detect/yolov8m_custom/weights/best.pt')
metrics = model.val()

### Detect demo

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# replace with the weights you want
!yolo task=detect \
mode=predict \
model="runs/detect/yolov8n_custom/weights/best.pt" \
source="UFDD_val/images/haze/haze_02101.jpg" \
show=True \
imgsz=640 \
name=yolov8n_inference \
show_labels=False

## **YOLOv5 Training**
### Just for YOLOv5 vs YOLOv8 Comparison

### Training

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks
%cd ../

In [ ]:
# Train YOLOv5n
%cd yolov5
!python train.py --img 640 --batch 64 --epochs 200 --data "../UFDD.yaml" --weights yolov5n.pt #adjust the hyperparameter youself
%cd ../

### Demo

In [ ]:
%cd yolov5
!python detect.py --weights "runs/train/exp/weights/best.pt" --img 640 --conf 0.25 --source "../UFDD_val/images/haze/haze_02101.jpg"
%cd ../

## **OpenVINO for YOLOv8**



In [ ]:
!pip install -q "openvino-dev>=2023.0.0" "nncf>=2.5.0"
!pip install -q "ultralytics==8.0.146" "onnx==1.14.1"

In [ ]:
# your pretrained model
experiment = 'yolov8n_custom'
det_model = YOLO(f"runs/detect/{experiment}/weights/best.pt")

# export model to openvino format (using prebuilt export function in yolov8)
openvino_path = f"runs/detect/{experiment}/weights/best_openvino_model/best.xml"
if not os.path.isfile(openvino_path):
    det_model.export(format="openvino", dynamic=True, half=False)

# **OpenVINO: Post Training Optimization**
Potential future add-ons </br>
Reference
*    [Object Detection Quantization](https://docs.openvino.ai/2022.2/notebooks/111-detection-quantization-with-output.html)

In [ ]:
from ultralytics.yolo.utils import DEFAULT_CFG
from ultralytics.yolo.cfg import get_cfg
from ultralytics.yolo.data.utils import check_det_dataset

CFG_PATH = 'UFDD.yaml'
args = get_cfg(cfg=DEFAULT_CFG)
args.data = str(CFG_PATH)

In [ ]:
det_model

In [ ]:
det_model = YOLO("/content/runs/detect/yolov8n_custom/weights/best.pt")

In [ ]:
det_validator = det_model.ValidatorClass(args=args)

In [ ]:
import nncf  # noqa: F811
from typing import Dict


def transform_fn(data_item:Dict):
    """
    Quantization transform function. Extracts and preprocess input data from dataloader item for quantization.
    Parameters:
       data_item: Dict with data item produced by DataLoader during iteration
    Returns:
        input_tensor: Input data for quantization
    """
    input_tensor = det_validator.preprocess(data_item)['img'].numpy()
    return input_tensor


quantization_dataset = nncf.Dataset(det_data_loader, transform_fn)

In [ ]:
from openvino.runtime import serialize
int8_model_det_path = f"/content/runs/detect/{experiment}/weights/best_openvino_model/best_int8.xml"
print(f"Quantized detection model will be saved to {int8_model_det_path}")
serialize(quantized_det_model, str(int8_model_det_path))